In [1]:
import nltk
import re
import codecs
import csv
import os
from sklearn import metrics

path = 'D:\Dropbox\ongoing\chrisIntervention'

In [2]:
# read cohort info from the CSV and save them with the sbj ID

intvIdx = 'Ex1'

category = {}
with open(os.path.join(path, intvIdx + '_key.csv')) as f:
    sbjData = csv.reader(f)
    next(sbjData) # skip the header
    for row in sbjData:
        category[row[0]] = int(row[1])

subjectId = category.keys()        
        
#print(subjectId)
#print()
print('# of cohort 1: ', sum(1 for sbj in category if category[sbj] == 1))
print('# of cohort 2: ', sum(1 for sbj in category if category[sbj] == 2))


# of cohort 1:  148
# of cohort 2:  197


In [3]:
# build the dictionary based on the training set
# this is just to check if it is working
# we should build the dictionary for each leave-one-out test

def make_dictionary(subjectPool, critNumSbj):
    allWords = []
    for sbj in subjectPool:
        sbjWords = []
        for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
            line = line.strip()
            line = line.lower()
            tokens = nltk.wordpunct_tokenize(line)
            text = nltk.Text(tokens)
            words = [w for w in text if re.search('[a-z]+',w) and len(w)>2]
            sbjWords.extend(words)
        # when making dictionary, make the words from a document unique
        sbjWords = list(set(sbjWords))
        allWords.extend(sbjWords)
    
    # once we make list of all words, we count the words
    cntWords = nltk.FreqDist(allWords)
    #for ii in range(21,0,-2):
    #    print('# words with freq >', ii, ' : ', sum(1 for word in cntWords if cntWords[word] > ii))
    
    # we go with the words that came from at least N different people (arbitrary) 
    # we can change and see what happens ...
    return [word for word in cntWords if cntWords[word] > critNumSbj]

dictionary = make_dictionary(subjectId, 5)

print(dictionary)

['spending', 'comes', 'learn', 'around', 'going', 'trouble', 'first', 'sports', 'know', 'creative', 'make', 'should', 'mom', 'now', 'stay', 'try', 'about', 'own', 'team', 'take', 'other', 'helps', 'others', 'only', 'new', 'able', 'these', 'music', 'into', 'draw', 'them', 'live', 'parents', 'play', 'want', 'something', 'show', 'when', 'while', 'sing', 'there', 'home', 'big', 'values', 'everyone', 'games', 'are', 'important', 'dance', 'cause', 'doing', 'someone', 'say', 'their', 'were', 'feeling', 'basketball', 'more', 'without', 'really', 'active', 'living', 'any', 'hop', 'has', 'gives', 'together', 'through', 'help', 'freinds', 'enjoying', 'people', 'see', 'from', 'social', 'never', 'even', 'dad', 'different', 'fun', 'very', 'talk', 'day', 'matter', 'today', 'smile', 'side', 'work', 'express', 'long', 'school', 'game', 'than', 'your', 'listing', 'back', 'sister', 'most', 'sometimes', 'healthy', 'well', 'laugh', 'better', 'have', 'second', 'could', 'songs', 'you', 'way', 'after', 'had',

In [4]:
# we define a function that tunrs text into a list of tokens

def extract_feature(text, dictionary):
    text = text.strip()
    text = text.lower()
    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)
    feature = {}
    for word in dictionary:
        feature[word] = (word in text)
    return feature

print('nltk output: ', nltk.wordpunct_tokenize('You think you can do this one, Kate?'))
print(extract_feature('You think you can do this one, Kate?', dictionary))

nltk output:  ['You', 'think', 'you', 'can', 'do', 'this', 'one', ',', 'Kate', '?']
{'need': False, 'really': False, 'just': False, 'without': False, 'than': False, 'learn': False, 'them': False, 'inportant': False, 'two': False, 'live': False, 'dad': False, 'parents': False, 'active': False, 'your': False, 'group': False, 'living': False, 'care': False, 'any': False, 'play': False, 'spending': False, 'comes': False, 'done': False, 'yourself': False, 'religious': False, 'bad': False, 'gives': False, 'together': False, 'work': False, 'sister': False, 'want': False, 'still': False, 'value': False, 'one': True, 'something': False, 'anything': False, 'years': False, 'around': False, 'who': False, 'going': False, 'nice': False, 'show': False, 'when': False, 'keep': False, 'hang': False, 'while': False, 'most': False, 'trouble': False, 'nothing': False, 'freinds': False, 'think': True, 'enjoying': False, 'sing': False, 'where': False, 'sometimes': False, 'people': False, 'see': False, 'first

In [5]:
# leave-one-out classification

predLabel = []
trueLabel = []
currSbj = 1;

for sbj in subjectId:
    # sbj is being tested, others are used to train the classifier
    trainSbj = list(subjectId)
    trainSbj.remove(sbj)
    # create the dictionary
    trainDict = make_dictionary(trainSbj, 5)
    trainText = []
    # create the training set
    for tSbj in trainSbj:
        tmpText = ''
        for line in codecs.open(os.path.join(path, intvIdx, tSbj + '.txt'), 'r' , 'utf-8'):
            tmpText = tmpText + ' ' + line
        trainText.append([extract_feature(tmpText, trainDict), category[tSbj]])
    # train classifier
    classifier = nltk.NaiveBayesClassifier.train(trainText)

    # now the test set
    trueLabel.append( category[sbj] )
    testText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        testText = testText + ' ' + line
    # predict!
    currPred = classifier.classify(extract_feature(testText, trainDict))
    predLabel.append( currPred )
    
    print(str(currSbj) + ' (' + sbj + '): pred - ' + str(currPred) + ', actual - ' + str(category[sbj]) )
    currSbj = currSbj + 1

print('done. ')


1 (620136_1): pred - 1, actual - 2
2 (920108_1): pred - 2, actual - 2
3 (110035_1): pred - 1, actual - 1
4 (211002_1): pred - 1, actual - 1
5 (720059_1): pred - 2, actual - 2
6 (1020097_1): pred - 2, actual - 2
7 (110004_1): pred - 2, actual - 1
8 (120016_1): pred - 1, actual - 2
9 (320075_1): pred - 2, actual - 2
10 (510109_1): pred - 1, actual - 1
11 (220002_1): pred - 2, actual - 2
12 (110081_1): pred - 2, actual - 1
13 (220063_1): pred - 2, actual - 2
14 (220031_1): pred - 2, actual - 2
15 (210519_1): pred - 2, actual - 1
16 (1020006_1): pred - 2, actual - 2
17 (810712_1): pred - 2, actual - 1
18 (1110111_1): pred - 2, actual - 1
19 (520044_1): pred - 2, actual - 2
20 (120084_1): pred - 2, actual - 2
21 (1110302_1): pred - 2, actual - 1
22 (1010327_1): pred - 2, actual - 1
23 (211105_1): pred - 1, actual - 1
24 (610202_1): pred - 1, actual - 1
25 (910318_1): pred - 1, actual - 1
26 (220117_1): pred - 2, actual - 2
27 (110070_1): pred - 2, actual - 1
28 (910410_1): pred - 1, actual 

In [6]:
# summarize the results

print(metrics.classification_report(trueLabel, predLabel))


             precision    recall  f1-score   support

          1       0.72      0.55      0.62       148
          2       0.71      0.84      0.77       197

avg / total       0.72      0.72      0.71       345



In [7]:
# ok, it is POSSIBLE TO PREDICT cohort from the writings.
# now, show me the informative features

dictionary = make_dictionary(subjectId, 5)
print(len(dictionary))

allText = []

for sbj in subjectId:
    tmpText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        tmpText = tmpText + ' ' + line
        allText.append([extract_feature(tmpText, dictionary), category[sbj]])

# train classifier based on all information
classifier = nltk.NaiveBayesClassifier.train(allText)


266


In [8]:
classifier.show_most_informative_features(100)


Most Informative Features
                  values = True                1 : 2      =     12.8 : 1.0
              everything = True                1 : 2      =     12.8 : 1.0
                   never = True                1 : 2      =      5.9 : 1.0
                     old = True                1 : 2      =      5.8 : 1.0
                    into = True                1 : 2      =      5.8 : 1.0
                     now = True                1 : 2      =      5.6 : 1.0
                     see = True                1 : 2      =      5.6 : 1.0
                    dont = True                1 : 2      =      5.1 : 1.0
                   angry = True                1 : 2      =      4.9 : 1.0
                 writing = True                1 : 2      =      4.9 : 1.0
                     rap = True                1 : 2      =      4.9 : 1.0
                    same = True                1 : 2      =      4.9 : 1.0
                    stay = True                1 : 2      =      4.9 : 1.0